In [ ]:
%pip install pandas numpy matplotlib plotly seaborn openpyxl

In [ ]:
import os
import pandas as pd


def get_egresos_csv_files(root_path):
    csv_files = [
        os.path.join(root_path, file)
        for file in os.listdir(root_path)
        if file.endswith(".csv")
    ]
    return sorted(csv_files)


egresos_root_path = "../../datasets/egresos-hospitalarios"
egresos_folders = sorted(os.listdir(egresos_root_path))
egresos_csvs = get_egresos_csv_files(egresos_root_path)

In [ ]:
egresos_df = pd.DataFrame()

for csv_file in egresos_csvs:
    egresos = pd.read_csv(
        csv_file,
        delimiter=";",
        converters={
            "CONDICION_EGRESO": lambda v: True if v == "2" else False,
            "INTERV_Q": lambda v: True if v == "1" else False,
            "PROCED": lambda v: True if v == "1" else False,
        },
    )
    egresos.columns = egresos.columns.str.lower()
    egresos.rename(
        columns={
            "ano_egreso": "año",
            "diag1": "diagnostico_principal",
            "diag2": "causa_externa",
            "dias_estada": "dias_hospitalizacion",
            "interv_q": "intervencion_quirurgica",
            "proced": "procedimiento",
            "condicion_egreso": "egreso_muerte",
        },
        inplace=True,
    )

    if "glosa_interv_q_ppal" in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("egreso_muerte") + 1,
            "intervencion_quirurgica",
            None,
        )
        egresos["intervencion_quirurgica"] = egresos["glosa_interv_q_ppal"].notna()

    if "glosa_proced_ppal" in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("intervencion_quirurgica") + 1,
            "procedimiento",
            None,
        )
        egresos["procedimiento"] = egresos["glosa_proced_ppal"].notna()

    if "intervencion_quirurgica" not in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("egreso_muerte") + 1,
            "intervencion_quirurgica",
            None,
        )

    if "procedimiento" not in egresos.columns:
        egresos.insert(
            egresos.columns.get_loc("intervencion_quirurgica") + 1,
            "procedimiento",
            None,
        )

    egresos.drop(
        columns=[
            "pertenencia_establecimiento_salud",
            "pertenencia_establecimiento_salu",
            "prevision",
            "glosa_prevision",
            "region_residencia",
            "glosa_region_residencia",
            "sexo",
            "glosa_interv_q_ppal",
            "glosa_proced_ppal",
        ],
        inplace=True,
        errors="ignore",
    )
    egresos["procedimiento"] = egresos["procedimiento"].astype(bool)
    egresos["intervencion_quirurgica"] = egresos["intervencion_quirurgica"].astype(bool)
    egresos_df = pd.concat([egresos_df, egresos], ignore_index=True)

print(egresos_df["egreso_muerte"].value_counts())
print(egresos_df["intervencion_quirurgica"].value_counts())
print(egresos_df["procedimiento"].value_counts())
print(egresos_df.info())

In [ ]:
egresos_df.head()

In [ ]:
egresos_df[["diagnostico_principal"]].value_counts(dropna=False)

In [ ]:
diagnosticos = pd.read_excel(
    "../../datasets/egresos-hospitalarios/diccionario.xlsx",
    sheet_name="codigo CIE-10",
    skiprows=8,
)
diagnosticos.columns = diagnosticos.columns.str.lower().str.replace(" ", "_")
diagnosticos.head()

In [ ]:
print(len(diagnosticos["glosa_subcategoria"].unique()))
print(len(diagnosticos["glosa_grupo_"].unique()))

In [ ]:
egresos_diag_df = pd.merge(
    egresos_df,
    diagnosticos.add_prefix("diagnostico_"),
    left_on="diagnostico_principal",
    right_on="diagnostico_codigo_subcategoria",
)
print(egresos_diag_df.info())
egresos_diag_df.head()

In [ ]:
egresos_diag_df[["causa_externa"]].isna().value_counts(dropna=False)

In [ ]:
egresos_diag_causa_df = pd.merge(
    egresos_diag_df,
    diagnosticos.add_prefix("causa_externa_"),
    left_on="causa_externa",
    right_on="causa_externa_codigo_subcategoria",
    how="left",
)

print(egresos_diag_causa_df.info())
print(egresos_diag_causa_df.head())

egresos_diag_causa_df["diagnostico_principal"] = egresos_diag_causa_df[
    "diagnostico_glosa_subcategoria"
]
egresos_diag_causa_df["causa_externa"] = egresos_diag_causa_df[
    "causa_externa_glosa_subcategoria"
]

egresos_diag_causa_df.rename(
    columns={
        "diagnostico_glosa_grupo_": "diagnostico_glosa_grupo",
        "causa_externa_glosa_grupo_": "causa_externa_glosa_grupo",
    },
    inplace=True,
)

In [ ]:
print("isna?", egresos_diag_causa_df[["diagnostico_principal"]].isna().value_counts())
print("isna?", egresos_diag_causa_df[["causa_externa"]].isna().value_counts())

In [ ]:
egresos_diag_causa_df.drop(
    columns=[
        "diagnostico_codigo_subcategoria",
        "diagnostico_glosa_subcategoria",
        "diagnostico_capitulo",
        "diagnostico_codigo_grupo_",
        "diagnostico_codigo_categoria",
        "causa_externa_codigo_subcategoria",
        "causa_externa_glosa_subcategoria",
        "causa_externa_capitulo",
        "causa_externa_codigo_grupo_",
        "causa_externa_codigo_categoria",
    ],
    inplace=True,
    errors="ignore",
)

In [ ]:
egresos_diag_causa_df.info()

In [ ]:
egresos_diag_causa_df[["comuna_residencia"]].isna().value_counts()

In [ ]:
%run ../load_comunas_servicios.py "../../datasets/comunas_por_zona.csv"

comunas_servicios_df.rename(columns={"zona_id": "capitulo", "zona": "nombre_capitulo"}, inplace=True)
print(comunas_servicios_df.info())
comunas_servicios_df.head()

In [ ]:
comunas_servicios_df[comunas_servicios_df["comuna"] == "Chaitén"]

In [ ]:
egresos_diag_causa_df = pd.merge(
    egresos_diag_causa_df,
    comunas_servicios_df[["capitulo", "nombre_capitulo", "comuna_id"]],  # type: ignore
    left_on="comuna_residencia",
    right_on="comuna_id",
    how="left",
)

In [ ]:
print(egresos_diag_causa_df[["capitulo"]].isna().value_counts())
egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()]

In [ ]:
print(
    egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
        "comuna_residencia"
    ].unique()
)
print(
    egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
        "glosa_comuna_residencia"
    ].unique()
)

In [ ]:
missing_capitulos = egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
    ["comuna_residencia", "glosa_comuna_residencia"]
]
missing_capitulos.value_counts()

In [ ]:
egresos_diag_causa_df = egresos_diag_causa_df[
    ~egresos_diag_causa_df["comuna_residencia"].isin([88888, 99999])
]
missing_capitulos = egresos_diag_causa_df[egresos_diag_causa_df["capitulo"].isna()][
    ["comuna_residencia", "glosa_comuna_residencia"]
]
missing_capitulos.value_counts()

In [ ]:
egresos_diag_causa_df.drop(
    columns=["comuna_id", "comuna_residencia", "glosa_comuna_residencia"],
    inplace=True,
    errors="ignore",
)

egresos_diag_causa_df.head()

In [ ]:
egresos_diag_causa_df.info()

In [ ]:
egresos_diag_causa_df = egresos_diag_causa_df[
    [
        "año",
        "capitulo",
        "nombre_capitulo",
        "grupo_edad",
        "etnia",
        "glosa_pais_origen",
        "diagnostico_principal",
        "diagnostico_glosa_categoria",
        "diagnostico_glosa_grupo",
        "diagnostico_glosa_capitulo",
        "dias_hospitalizacion",
        "egreso_muerte",
        "causa_externa",
        "causa_externa_glosa_categoria",
        "causa_externa_glosa_grupo",
        "causa_externa_glosa_capitulo",
        "intervencion_quirurgica",
        "procedimiento",
    ]
]
egresos_diag_causa_df

In [ ]:
egresos_diag_causa_df.drop(
    columns=["grupo_edad", "etnia", "glosa_pais_origen"], inplace=True
)

In [ ]:
egresos_diag_causa_df.info()

In [ ]:
DIAGNOSTICO_VARIABLE = "diagnostico_glosa_grupo"

egresos_diag_causa_df.query(
    f"año == 2023 and capitulo == 53 and {DIAGNOSTICO_VARIABLE} == 'Úlcera gástrica'"
)["dias_hospitalizacion"].count()

In [ ]:
grouped_egresos_df = (
    egresos_diag_causa_df.groupby(
        ["año", "capitulo", "nombre_capitulo", DIAGNOSTICO_VARIABLE]
    )
    .agg({"dias_hospitalizacion": ["count", "sum", "mean"]})
    .reset_index()
)

grouped_egresos_df.columns = [
    "_".join(col) if col[1] else col[0] for col in grouped_egresos_df.columns
]

grouped_egresos_df.rename(
    columns={
        "dias_hospitalizacion_count": "cantidad_hospitalizaciones",
        "dias_hospitalizacion_sum": "dias_hospitalizacion_total",
        "dias_hospitalizacion_mean": "dias_hospitalizacion_promedio",
    },
    inplace=True,
    errors="ignore",
)
grouped_egresos_df

In [ ]:
grouped_egresos_df.sort_values(
    by=["año", "capitulo", "nombre_capitulo", "cantidad_hospitalizaciones"],
    ascending=[True, True, True, False],
).query("capitulo == 21")

In [ ]:
K = 5


def get_top_k_for(k, variables, df):
    result_df = pd.DataFrame()

    for variable in variables:
        top_k_df = (
            df.sort_values(
                by=["año", "capitulo", "nombre_capitulo", variable],
                ascending=[True, True, True, False],
            )
            .groupby(["año", "capitulo", "nombre_capitulo"])[
                [
                    "año",
                    "capitulo",
                    "nombre_capitulo",
                    DIAGNOSTICO_VARIABLE,
                    variable,
                ]
            ]
            .head(k)
            .reset_index(drop=True)
        )

        if result_df.empty:
            result_df = top_k_df.rename(
                columns={
                    variable: f"top_{k}_{variable}",
                    DIAGNOSTICO_VARIABLE: f"top_{k}_{variable}_diagnostico",
                }
            )
        else:
            top_k_variable_name = f"top_{k}_{variable}"
            new_columns = top_k_df.rename(
                columns={
                    variable: top_k_variable_name,
                    DIAGNOSTICO_VARIABLE: f"{top_k_variable_name}_diagnostico",
                }
            ).iloc[:, -2:]
            result_df = pd.concat([result_df, new_columns], axis=1)

    return result_df


top_k_grouped_df = get_top_k_for(
    K,
    [
        "cantidad_hospitalizaciones",
        "dias_hospitalizacion_total",
        "dias_hospitalizacion_promedio",
    ],
    grouped_egresos_df,
)

top_k_grouped_df

In [ ]:
hospitalizaciones_anio_servicio_df = (
    top_k_grouped_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=[f"top_{K}_cantidad_hospitalizaciones_diagnostico"],
        values=[f"top_{K}_cantidad_hospitalizaciones"],
    )
    .reset_index()
    .fillna(0)
)

hospitalizaciones_anio_servicio_df.columns = [
    "_".join(col).replace(" ", "_").replace(",", "").lower() if col[1] else col[0]
    for col in hospitalizaciones_anio_servicio_df.columns
]

hospitalizaciones_anio_servicio_df

In [ ]:
dias_hospitalizacion_total_anio_servicio_df = (
    top_k_grouped_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=[f"top_{K}_dias_hospitalizacion_total_diagnostico"],
        values=[f"top_{K}_dias_hospitalizacion_total"],
    )
    .reset_index()
    .fillna(0)
)

dias_hospitalizacion_total_anio_servicio_df.columns = [
    "_".join(col).replace(" ", "_").replace(",", "").lower() if col[1] else col[0]
    for col in dias_hospitalizacion_total_anio_servicio_df.columns
]

dias_hospitalizacion_total_anio_servicio_df

In [ ]:
dias_hospitalizacion_promedio_anio_servicio_df = (
    top_k_grouped_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=[f"top_{K}_dias_hospitalizacion_promedio_diagnostico"],
        values=[f"top_{K}_dias_hospitalizacion_promedio"],
    )
    .reset_index()
    .fillna(0)
)

dias_hospitalizacion_promedio_anio_servicio_df.columns = [
    "_".join(col).replace(" ", "_").replace(",", "").lower() if col[1] else col[0]
    for col in dias_hospitalizacion_promedio_anio_servicio_df.columns
]

dias_hospitalizacion_promedio_anio_servicio_df

In [ ]:
final_df = pd.concat(
    [
        hospitalizaciones_anio_servicio_df,
        dias_hospitalizacion_total_anio_servicio_df.iloc[:, 3:],
        dias_hospitalizacion_promedio_anio_servicio_df.iloc[:, 3:],
    ],
    axis=1,
)
final_df

In [ ]:
final_df.to_csv("../../datasets/egresos_hospitalarios.csv", index=False)